# **Práctica Flujos de Datos**
- Clasificación.
- Detección de Concept Drift.
- Agrupamiento.
- Tratamiento de Texto.
- Ensembles.

In [35]:
import pandas as pd
import numpy as np
import river
from river import datasets
from river import tree
from river import metrics
#from river import preprocessing
from river import evaluate
from river import drift
from river import preprocessing, cluster
import math
import time

## **Base de Datos**
https://www.kaggle.com/datasets/mariumfaheem666/spam-sms-classification-using-nlp

Se ha elegido un conjunto de datos utilizado para la detección de SPAM, en este la primera columna indica si el SMS recibido es SPAM o no y la siguiente contiene el texto del SMS. A partir de la segunda columna del texto, se han obtenido los siguientes atributos: el número de caracteres del SMS, el número de palabras, número de caracteres alfanuméricos, número de caracteres no alfanuméricos, número de símbolos de divisas, número de mayúsculas, de exclamaciones, de interrogaciones y de urls en cada mensaje. Estas características se han elegido ya que a nuestro parecer son importantes a la hora de decidir si un SMS es SPAM o no.

In [2]:
# Carga de Datos
df = pd.read_csv('spam_SMS_ampliado.csv')
df.columns

Index(['spam', 'text', 'num_caracteres', 'num_palabras', 'num_alfabeticos',
       'num_numericos', 'num_no_alfanum', 'num_divisas', 'num_mayusculas',
       'num_exclamaciones', 'num_interrogaciones', 'num_urls'],
      dtype='object')

In [30]:
df.head(5)

,spam,text,num_caracteres,num_palabras,num_alfabeticos,num_numericos,num_no_alfanum,num_divisas,num_mayusculas,num_exclamaciones,num_interrogaciones,num_urls
0,0,"Go until jurong point, crazy.. Available only ...",111,20,83,0,28,0,3,0,0,0
1,0,Ok lar... Joking wif u oni...,29,6,18,0,11,0,2,0,0,0
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,28,97,25,33,0,10,0,0,0
3,0,U dun say so early hor... U c already then say...,49,11,33,0,16,0,2,0,0,0
4,0,"Nah I don't think he goes to usf, he lives aro...",61,13,47,0,14,0,2,0,0,0


In [33]:
len(df)

5574

## Clasificación

In [8]:
# crear, entrenar y evaluar modelo
model_standard = tree.HoeffdingTreeClassifier()
metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_standard.predict_one(x)  # realiza una predicción
    model_standard.learn_one(x, y)          # entrena el modelo con un ejemplo
    metric.update(y, y_pred)       # actualiza la métrica
    
     
print('Standard', metric)

Standard F1: 86.65%


In [9]:
# crear, entrenar y evaluar modelo
model_adaptive = tree.HoeffdingAdaptiveTreeClassifier()
#metric = metrics.Accuracy()
metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_adaptive.predict_one(x)  # realiza una predicción
    model_adaptive.learn_one(x, y)          # entrena el modelo con un ejemplo
    metric.update(y, y_pred)       # actualiza la métrica
     
print('Adaptive', metric)

Adaptive F1: 86.62%


In [26]:
# crear, entrenar y evaluar modelo
model_extreme = tree.ExtremelyFastDecisionTreeClassifier(grace_period=120)
#metric = metrics.Accuracy()
metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_extreme.predict_one(x)  # realiza una predicción
    model_extreme.learn_one(x, y)          # entrena el modelo con un ejemplo
    metric.update(y, y_pred)       # actualiza la métrica
     
print('Extremely Fast', metric)

Extremely Fast F1: 85.23%


## Detección de Concept Drift

In [6]:
# detectores de drift
kswin = drift.KSWIN()
eddm = drift.EDDM()
ddm = drift.DDM()

### Concept Drift en HoeffdinTree

In [11]:
# crear, entrenar y evaluar modelo
model_standard = tree.HoeffdingTreeClassifier()
#metric = metrics.Accuracy()
metric = metrics.F1()

# detectores de drift
kswin = drift.KSWIN()
eddm = drift.EDDM()
ddm = drift.DDM()

for index, row in df.iterrows():
    x = {
        'num_caracteres': row['num_caracteres'],
        'num_palabras': row['num_palabras'],
        'num_alfabeticos': row['num_alfabeticos'],
        'num_numericos': row['num_numericos'],
        'num_no_alfanum': row['num_no_alfanum'],
        'num_divisas': row['num_divisas'],
        'num_mayusculas': row['num_mayusculas'],
        'num_exclamaciones': row['num_exclamaciones'],
        'num_interrogaciones': row['num_interrogaciones'],
        'num_urls': row['num_urls']
    }

    y = row['spam'] 
    y_pred = model_standard.predict_one(x)  # realiza una predicción
    model_standard.learn_one(x, y)          # entrena el modelo con un ejemplo
    metric.update(y, y_pred)       # actualiza la métrica
    # actualizar los detectores según haya predicho bien o mal
    eddm.update(int(y == y_pred))
    ddm.update(int(y == y_pred))
    kswin.update(int((x['num_mayusculas'])))   # en principio, la variable más representativa
    
    # Comprobar si DDM detectó Model Drift:
    if eddm.drift_detected: 
        print(f"EDDM - Model drift detectado en el ejemplo {index} ({metric})")
    
    # Comprobar si EDDM detectó Model Drift:
    if ddm.drift_detected:# .drift_detected: 
        print(f"DDM - Model drift detectado en el ejemplo {index} ({metric})")

    # Comprobar si ADWIN detectó Data Drift:
    if kswin.drift_detected:
        print(f"KSWIN - Data drift detectado en el ejemplo {index} ({metric})")

print('Standard', metric)

EDDM - Concept drift detectado en el ejemplo 48 (F1: 14.29%)
EDDM - Concept drift detectado en el ejemplo 111 (F1: 16.00%)
EDDM - Concept drift detectado en el ejemplo 174 (F1: 53.57%)
EDDM - Concept drift detectado en el ejemplo 224 (F1: 52.46%)
EDDM - Concept drift detectado en el ejemplo 297 (F1: 59.26%)
EDDM - Concept drift detectado en el ejemplo 389 (F1: 68.47%)
DDM - Concept drift detectado en el ejemplo 440 (F1: 71.43%)
DDM - Concept drift detectado en el ejemplo 471 (F1: 72.73%)
EDDM - Concept drift detectado en el ejemplo 478 (F1: 73.13%)
DDM - Concept drift detectado en el ejemplo 502 (F1: 73.91%)
EDDM - Concept drift detectado en el ejemplo 557 (F1: 75.64%)
KSWIN - Data drift detectado en el ejemplo 2043 (F1: 86.32%)
KSWIN - Data drift detectado en el ejemplo 2469 (F1: 86.13%)
KSWIN - Data drift detectado en el ejemplo 3346 (F1: 85.94%)
Standard F1: 86.65%


### Concept Drift en HoeffdingAdaptiveTree

In [12]:
# crear, entrenar y evaluar modelo
model_adaptive = tree.HoeffdingAdaptiveTreeClassifier()
metric = metrics.F1()

# detectores de drift
kswin = drift.KSWIN()
eddm = drift.EDDM()
ddm = drift.DDM()

for index, row in df.iterrows():
    x = {
        'num_caracteres': row['num_caracteres'],
        'num_palabras': row['num_palabras'],
        'num_alfabeticos': row['num_alfabeticos'],
        'num_numericos': row['num_numericos'],
        'num_no_alfanum': row['num_no_alfanum'],
        'num_divisas': row['num_divisas'],
        'num_mayusculas': row['num_mayusculas'],
        'num_exclamaciones': row['num_exclamaciones'],
        'num_interrogaciones': row['num_interrogaciones'],
        'num_urls': row['num_urls']
    }

    y = row['spam'] 
    y_pred = model_adaptive.predict_one(x)  # realiza una predicción
    model_adaptive.learn_one(x, y)          # entrena el modelo con un ejemplo
    metric.update(y, y_pred)       # actualiza la métrica
    # actualizar los detectores según haya predicho bien o mal
    eddm.update(int(y == y_pred))
    ddm.update(int(y == y_pred))
    kswin.update(int((x['num_mayusculas'])))   # en principio, la variable más representativa
    
    # Comprobar si DDM detectó Model Drift:
    if eddm.drift_detected: 
        print(f"EDDM - Model drift detectado en el ejemplo {index} ({metric})")
    
    # Comprobar si EDDM detectó Model Drift:
    if ddm.drift_detected:# .drift_detected: 
        print(f"DDM - Model drift detectado en el ejemplo {index} ({metric})")

    # Comprobar si ADWIN detectó Data Drift:
    if kswin.drift_detected:
        print(f"KSWIN - Data drift detectado en el ejemplo {index} ({metric})")

print('Adaptive', metric)

EDDM - Concept drift detectado en el ejemplo 48 (F1: 14.29%)
EDDM - Concept drift detectado en el ejemplo 224 (F1: 61.29%)
EDDM - Concept drift detectado en el ejemplo 297 (F1: 65.85%)
DDM - Concept drift detectado en el ejemplo 363 (F1: 70.59%)
EDDM - Concept drift detectado en el ejemplo 389 (F1: 73.21%)
DDM - Concept drift detectado en el ejemplo 394 (F1: 73.21%)
EDDM - Concept drift detectado en el ejemplo 478 (F1: 77.04%)
EDDM - Concept drift detectado en el ejemplo 557 (F1: 78.98%)
KSWIN - Data drift detectado en el ejemplo 2472 (F1: 86.88%)
KSWIN - Data drift detectado en el ejemplo 3436 (F1: 86.57%)
KSWIN - Data drift detectado en el ejemplo 4619 (F1: 87.31%)
Adaptive F1: 87.62%


### Concept Drift en ExtremelyFastDecisionTree

In [25]:
# crear, entrenar y evaluar modelo
model_extreme = tree.ExtremelyFastDecisionTreeClassifier(grace_period=120)
metric = metrics.F1()

# detectores de drift
kswin = drift.KSWIN()
eddm = drift.EDDM()
ddm = drift.DDM()

for index, row in df.iterrows():
    x = {
        'num_caracteres': row['num_caracteres'],
        'num_palabras': row['num_palabras'],
        'num_alfabeticos': row['num_alfabeticos'],
        'num_numericos': row['num_numericos'],
        'num_no_alfanum': row['num_no_alfanum'],
        'num_divisas': row['num_divisas'],
        'num_mayusculas': row['num_mayusculas'],
        'num_exclamaciones': row['num_exclamaciones'],
        'num_interrogaciones': row['num_interrogaciones'],
        'num_urls': row['num_urls']
    }

    y = row['spam'] 
    y_pred = model_extreme.predict_one(x)  # realiza una predicción
    model_extreme.learn_one(x, y)          # entrena el modelo con un ejemplo
    metric.update(y, y_pred)       # actualiza la métrica
    # actualizar los detectores según haya predicho bien o mal
    eddm.update(int(y == y_pred))
    ddm.update(int(y == y_pred))
    kswin.update(int((x['num_mayusculas'])))   # en principio, la variable más representativa
    
    # Comprobar si DDM detectó Model Drift:
    if eddm.drift_detected: 
        print(f"EDDM - Model drift detectado en el ejemplo {index} ({metric})")
    
    # Comprobar si EDDM detectó Model Drift:
    if ddm.drift_detected:# .drift_detected: 
        print(f"DDM - Model drift detectado en el ejemplo {index} ({metric})")

    # Comprobar si ADWIN detectó Data Drift:
    if kswin.drift_detected:
        print(f"KSWIN - Data drift detectado en el ejemplo {index} ({metric})")

print('Extremely Fast', metric)

EDDM - Concept drift detectado en el ejemplo 48 (F1: 14.29%)
EDDM - Concept drift detectado en el ejemplo 111 (F1: 16.00%)
EDDM - Concept drift detectado en el ejemplo 161 (F1: 50.00%)
EDDM - Concept drift detectado en el ejemplo 216 (F1: 53.57%)
KSWIN - Data drift detectado en el ejemplo 376 (F1: 69.23%)
DDM - Concept drift detectado en el ejemplo 414 (F1: 69.64%)
EDDM - Concept drift detectado en el ejemplo 654 (F1: 79.14%)
EDDM - Concept drift detectado en el ejemplo 845 (F1: 82.17%)
EDDM - Concept drift detectado en el ejemplo 914 (F1: 83.10%)
KSWIN - Data drift detectado en el ejemplo 2043 (F1: 83.79%)
KSWIN - Data drift detectado en el ejemplo 3434 (F1: 85.22%)
KSWIN - Data drift detectado en el ejemplo 3566 (F1: 85.18%)
Extremely Fast F1: 85.23%


## Detección de Anomalías

In [36]:
# Crear un scaler para estandarizar los datos
scaler = preprocessing.StandardScaler()

# Inicializar K-Means con 5 clústeres
kmeans = cluster.KMeans(n_clusters=5, seed=42)

# Umbral para considerar un punto como anomalía basándonos en la distancia al centroide
threshold = 7.0

# Para almacenar el conteo de anomalías
anomalies_detected = 0

# Función para calcular la distancia euclidiana entre un punto y un centroide
def euclidean_distance(point, center):
    return math.sqrt(sum((point[feature] - center[feature]) ** 2 for feature in point))

# Iterar sobre cada fila del DataFrame para simular un flujo de datos
for index, row in df.iterrows():
    # Seleccionar solo las características numéricas relevantes
    features = {
        'num_caracteres': float(row['num_caracteres']),
        'num_palabras': float(row['num_palabras']),
        'num_alfabeticos': float(row['num_alfabeticos']),
        'num_numericos': float(row['num_numericos']),
        'num_no_alfanum': float(row['num_no_alfanum']),
        'num_divisas': float(row['num_divisas']),
        'num_mayusculas': float(row['num_mayusculas']),
        'num_exclamaciones': float(row['num_exclamaciones']),
        'num_interrogaciones': float(row['num_interrogaciones']),
        'num_urls': float(row['num_urls'])
    }

    # Actualizar el escalador con los nuevos datos
    scaler.learn_one(features)
    
    # Estandarizar las características
    features_scaled = scaler.transform_one(features)

    # Predecir el clúster más cercano
    cluster_id = kmeans.predict_one(features_scaled)

    # Obtener los centroides aprendidos hasta el momento
    centroids = kmeans.centers

    # Si el modelo ha aprendido suficientes centroides, calcular la distancia
    if centroids:
        # Obtener el centroide del clúster asignado
        centroid = centroids[cluster_id]

        # Calcular la distancia euclidiana al centroide
        distance_to_centroid = euclidean_distance(features_scaled, centroid)

        # Si la distancia es mayor que el umbral, lo consideramos una anomalía
        if distance_to_centroid > threshold:
            anomalies_detected += 1
            print(f"Anomalía detectada en {index} con distancia {distance_to_centroid:.2f}")

    # Actualizar el modelo KMeans con el nuevo punto
    kmeans.learn_one(features_scaled)

# Mostrar el número total de anomalías detectadas
print(f"Total de anomalías detectadas: {anomalies_detected}")

Anomalía detectada en 155 con distancia 7.88
Anomalía detectada en 164 con distancia 8.13
Anomalía detectada en 401 con distancia 7.40
Anomalía detectada en 445 con distancia 10.13
Anomalía detectada en 492 con distancia 7.36
Anomalía detectada en 793 con distancia 13.29
Anomalía detectada en 838 con distancia 8.99
Anomalía detectada en 1036 con distancia 7.09
Anomalía detectada en 1085 con distancia 25.62
Anomalía detectada en 1385 con distancia 7.57
Anomalía detectada en 1463 con distancia 8.08
Anomalía detectada en 1487 con distancia 7.12
Anomalía detectada en 1546 con distancia 8.04
Anomalía detectada en 1579 con distancia 10.47
Anomalía detectada en 1586 con distancia 7.78
Anomalía detectada en 1609 con distancia 9.89
Anomalía detectada en 1659 con distancia 8.96
Anomalía detectada en 1793 con distancia 7.29
Anomalía detectada en 1827 con distancia 7.81
Anomalía detectada en 1863 con distancia 9.58
Anomalía detectada en 1876 con distancia 7.57
Anomalía detectada en 2010 con distan

In [36]:
# Flujo de datos simulado
try:
    for index, row in df.iterrows():
        # Extrae características (X) y etiqueta (y)
        X = {
            'text': row['text'],
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

        y = row['spam'] 
        
        print(f"Datos recibidos: {X}, Etiqueta: {y}")
        
        # Aquí se aplicarían los algoritmos online
    
        time.sleep(0.5)  # Se espera medio segundo para simular el flujo de datos
        
except KeyboardInterrupt:
    print('Proceso detenido.')

Datos recibidos: {'text': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'num_caracteres': 111, 'num_palabras': 20, 'num_alfabeticos': 83, 'num_numericos': 0, 'num_no_alfanum': 28, 'num_divisas': 0, 'num_mayusculas': 3, 'num_exclamaciones': 0, 'num_interrogaciones': 0, 'num_urls': 0}, Etiqueta: 0
Datos recibidos: {'text': 'Ok lar... Joking wif u oni...', 'num_caracteres': 29, 'num_palabras': 6, 'num_alfabeticos': 18, 'num_numericos': 0, 'num_no_alfanum': 11, 'num_divisas': 0, 'num_mayusculas': 2, 'num_exclamaciones': 0, 'num_interrogaciones': 0, 'num_urls': 0}, Etiqueta: 0
Datos recibidos: {'text': "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'num_caracteres': 155, 'num_palabras': 28, 'num_alfabeticos': 97, 'num_numericos': 25, 'num_no_alfanum': 33, 'num_divisas': 0, 'num_mayusculas': 10, 'num_exclamaciones